In [1]:
from stackapi import StackAPI
import pandas as pd

In [2]:
import spotipy.util as util

username = 'ninahew'
client_id ='a758a8092e3b45949388c4123f79948f'
client_secret = 'c8025d9d9bb24c1c8f6156a5f66dcfe1'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

Couldn't read cache at: .cache-ninahew
Couldn't read cache at: .cache-ninahew


In [ ]:
#http://localhost:7777/callback?code=AQBrEDzH54knGVajsUUNPh_6ClqLpP9tYLlfQuLT2p26r3wF_hTAT24mLWZeApjbx2hesB0299ShGbVbuU0mdkya7HoyHyc31D0VCSULYOCF9T81GeH8YyL6RPCXRDT6O0meYuZAGS2ENCJftLq3hjoNDjouxiRUsFvAhgRA2P4PtQMGyMsittB0d9SxAUXh4f95mHMTkA4lL3A5AA

In [3]:
print(token)

BQD4K3ayyfealzU4q0jhWdRt4sSSTR9eQCjynbc6eY3wG4a6BYOW6HGNFnvIlzVGZhT0_47NSWgC7g6atljSBDD2yvs1h-Hj78BjYIIn5IEHbqMjbePOtTNYzHhIWUQFosdz6ZvqdP-pg9T67cg


In [15]:
#Get recent streaming history
import ast
from typing import List
from os import listdir
def get_streamings(path: str = '../data') -> List[dict]:
    
    files = ['../data' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings

In [16]:
#Get track IDs
import requests
def get_id(track_name: str, token: str) -> str:
    headers = {'Accept': 'application/json','Content-Type': 'application/json','Authorization': f'Bearer ' + token,}
    params = [('q', track_name),('type', 'track'),]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [17]:
#Get features of each track
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [18]:
streamings = get_streamings()
unique_tracks = list(set([streaming['trackName'] 
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        all_features[track] = features
        
with_features = []
for track_name, features in all_features.items():
    with_features.append({'name': track_name, **features})